# recommendation.py

In [4]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from surprise import SVD, Reader, Dataset

from azure.storage.blob import BlobServiceClient

In [1]:
# -------------------------------------------------------------------
# 1. Chargement des embeddings et du modèle CF (SVD)
# -------------------------------------------------------------------

# Ces variables globales seront initialisées lors du cold-start de la Function
embeddings = None           # numpy.ndarray, shape=(n_articles, 250)
meta = None                 # pandas.DataFrame avec colonnes ["article_id","category","publisher","words_count","created_at"]
nn_index = None             # NearestNeighbors indexé sur embeddings
cf_algo = None              # instance surprise.SVD entraînée
CF_RADIUS = 500             # taille du pool CF/CB par défaut

def load_embeddings_from_blob(blob_connection_string, container_name, blob_name, local_path):
    """
    Télécharge le fichier pickle des embeddings depuis un Blob Azure,
    puis charge le numpy.ndarray dans la variable globale `embeddings`.
    """
    from azure.storage.blob import BlobServiceClient

    # 1) Télécharger le blob sur un disque local
    blob_service = BlobServiceClient.from_connection_string(blob_connection_string)
    container = blob_service.get_container_client(container_name)
    blob_client = container.get_blob_client(blob_name)

    with open(local_path, "wb") as f_out:
        f_out.write(blob_client.download_blob().readall())

    # 2) Charger le pickle
    global embeddings
    with open(local_path, "rb") as f_in:
        embeddings = pickle.load(f_in)

    # 3) Supprimer le fichier temporaire si nécessaire
    # os.remove(local_path)


def load_meta_from_blob(blob_connection_string, container_name, blob_name, local_path):
    """
    Télécharge et charge le DataFrame meta (CSV).
    """
    from azure.storage.blob import BlobServiceClient

    blob_service = BlobServiceClient.from_connection_string(blob_connection_string)
    container = blob_service.get_container_client(container_name)
    blob_client = container.get_blob_client(blob_name)

    with open(local_path, "wb") as f_out:
        f_out.write(blob_client.download_blob().readall())

    global meta
    meta = pd.read_csv(local_path)
    # Si besoin, appliquer les mêmes transformations qu'en local
    meta["created_at"] = pd.to_datetime(
        meta["created_at_ts"], unit="s", origin="unix"
    )
    meta = meta.rename(columns={
        "category_id": "category",
        "publisher_id": "publisher"
    })[
        ["article_id","category","publisher","words_count","created_at"]
    ]


def index_embeddings():
    """
    Construit l'index NearestNeighbors sur `embeddings`.
    """
    global nn_index
    if embeddings is None:
        raise RuntimeError("Embeddings non chargés")
    nn_index = NearestNeighbors(
        n_neighbors=CF_RADIUS,
        metric="cosine",
        algorithm="auto"
    )
    nn_index.fit(embeddings)


def load_cf_model_from_blob(blob_connection_string, container_name, blob_name, local_path):
    """
    Télécharge un modèle Surprise (pickle) depuis Blob et le charge dans `cf_algo`.
    """
    from azure.storage.blob import BlobServiceClient

    blob_service = BlobServiceClient.from_connection_string(blob_connection_string)
    container = blob_service.get_container_client(container_name)
    blob_client = container.get_blob_client(blob_name)

    with open(local_path, "wb") as f_out:
        f_out.write(blob_client.download_blob().readall())

    global cf_algo
    with open(local_path, "rb") as f_in:
        cf_algo = pickle.load(f_in)


# -------------------------------------------------------------------
# 2. Fonctions utilitaires CB et CF
# -------------------------------------------------------------------

def build_user_profile(user_clicks):
    """
    user_clicks : liste d'article_id que l'utilisateur a lus (ou cliqué).
    Retourne la moyenne des embeddings correspondants (vecteur 250-d).
    """
    if embeddings is None or meta is None:
        raise RuntimeError("Embeddings ou meta manquants")

    # Récupérer les indices dans meta pour chaque article_id
    idxs = meta.index[meta["article_id"].isin(user_clicks)].tolist()
    if len(idxs) == 0:
        # Cold-start user : on renvoie un vecteur nul
        return np.zeros(embeddings.shape[1])

    user_embs = embeddings[idxs]
    return np.mean(user_embs, axis=0)


def score_cf_for_candidates(user_id, candidate_ids):
    """
    Pour un user_id et une liste d'article_id candidates, 
    retourne les scores CF bruts (algo.predict().est).
    """
    if cf_algo is None:
        raise RuntimeError("CF model non chargé")

    scores = []
    for iid in candidate_ids:
        pred = cf_algo.predict(uid=user_id, iid=iid, r_ui=None, verbose=False)
        scores.append(pred.est)
    return np.array(scores)


def normalize_minmax(array):
    """
    Min-max scaling pour ramener 'array' dans [0,1].
    Si min == max, on retourne un vecteur constant à 0.5.
    """
    mn = array.min()
    mx = array.max()
    if mx > mn:
        return (array - mn) / (mx - mn)
    else:
        return np.full_like(array, 0.5, dtype=float)


# -------------------------------------------------------------------
# 3. Fonction principale de recommandation hybride
# -------------------------------------------------------------------

def recommend_hybrid(
    user_id: int,
    user_clicks: list,
    k: int = 10,
    alpha: float = 0.6,
    total_candidates: int = CF_RADIUS
):
    """
    Renvoie un DataFrame pandas des k meilleurs articles pour user_id,
    en mélangeant CB et CF.

    - user_clicks : liste d'article_id (int) déjà cliqués par l'utilisateur
    - k           : nombre final de recommandations
    - alpha       : poids du CF (0-> uniquement CB, 1-> uniquement CF)
    - total_candidates : taille du pool brut de candidats à scorer
    """

    # 1) Construire le vecteur profil CB de l'utilisateur
    user_vec = build_user_profile(user_clicks)  # vecteur 250-dim

    # 2) Récupérer un pool de candidats CB via nearest neighbors
    distances, indices = nn_index.kneighbors([user_vec], n_neighbors=total_candidates)
    cand_idxs = indices[0]                # indices (rangées) dans meta
    sims_cb = 1.0 - distances[0]          # sim = 1 - distance_cosinus

    # 3) Construire le DataFrame de candidats
    article_ids = meta.iloc[cand_idxs]["article_id"].values.tolist()
    df_cand = pd.DataFrame({
        "article_id": article_ids,
        "score_cb": sims_cb
    })

    # 4) Calculer le score CF brut pour ces candidats
    cf_raw = score_cf_for_candidates(user_id, article_ids)
    df_cand["score_cf_raw"] = cf_raw

    # 5) Normaliser le score CF dans [0,1]
    df_cand["score_cf"] = normalize_minmax(df_cand["score_cf_raw"].values)

    # 6) Score hybride
    df_cand["score_hybrid"] = alpha * df_cand["score_cf"] + (1 - alpha) * df_cand["score_cb"]

    # 7) Trier par score_hybrid et prendre les top-k
    topk = (
        df_cand
        .sort_values("score_hybrid", ascending=False)
        .head(k)
        .merge(
            meta,
            on="article_id",
            how="left"
        )
    )

    # 8) Sélectionner/ordonner les colonnes à renvoyer
    return topk[[
        "article_id",
        "category",
        "publisher",
        "words_count",
        "created_at",
        "score_cb",
        "score_cf",
        "score_hybrid"
    ]].reset_index(drop=True)
